# Conversion de RAW a Reflectancia TOA google Earth Engine

URL: https://mt1.google.com/vt/lyrs=s&x=%7Bx%7D&y=%7By%7D&z=%7Bz%7D

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio SHP

In [2]:
import os
import glob

In [3]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP"

In [6]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [7]:
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP'

In [8]:
lista = glob.glob("*.shp")
lista

['Cuenca_Geo.shp', 'ZONA_geo.shp', 'Zona_Geografica.shp', 'Zona_UTM_19S.shp']

In [9]:
import geopandas as gpd

In [10]:
# cargar un shp
zona = gpd.read_file('ZONA_geo.shp')
zona

,id,geometry
0,1,"POLYGON ((-70.14663 -12.74826, -69.84347 -12.7..."


In [12]:
# Crear un featureCollecion mediante GeoJSON
ee_zona = ee.FeatureCollection(zona.__geo_interface__)

In [13]:
type(ee_zona)

ee.featurecollection.FeatureCollection

## Landsat 8 OLI TIRS RAW

In [14]:
# Coleccion de Landsat 8 OLI TIRS RAW
L8_RAW = ee.ImageCollection("LANDSAT/LC08/C01/T1")\
        .filterDate('2020-01-01','2021-06-05')\
        .filterMetadata('CLOUD_COVER', 'less_than',20)\
        .filterBounds(ee_zona)

In [15]:
# Conteo de imagenes
count = L8_RAW.size()
print("Cantidad de imagenes L8_RAW:", count.getInfo())

Cantidad de imagenes L8_RAW: 3


In [19]:
# Imprimir la lista de ID Imagenes
catalogo_L8 = L8_RAW.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()

In [22]:
print("ID catalogo_L8: ", catalogo_L8)

ID catalogo_L8:  ['LC08_003069_20200420', 'LC08_003069_20200607', 'LC08_003069_20200911']


In [30]:
# Importar imagen Landsat 8 
image = ee.Image("LANDSAT/LC08/C01/T1" + "/" + catalogo_L8[0])

In [31]:
# Consultar bandas
image.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [32]:
# Simbologia RAW de ND 16bits
viz_Raw = {
    'bands': ['B6', 'B5', 'B4'],
    0,
    'max': 18000'min': 600
}

In [33]:
geometria = image.geometry()

In [34]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [35]:
# Visualizacion de imagen
Map.centerObject(geometria, 8)
Map.addLayer(image, viz_Raw, 'L8 raw')
Map

Map(center=[-13.014742673841472, -70.46287583988538], controls=(WidgetControl(options=['position', 'transparen…

## Conversion de RAW a Radiancia

In [36]:
# Conversion de RAW a Radiancia
radiance = ee.Algorithms.Landsat.calibratedRadiance(image)

In [37]:
# Simbologia Radiancia
viz_Rad  = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 5,
    'max': 100
}

In [38]:
# Visualizacion de imagen
Map.addLayer(radiance, viz_Rad, 'radiance')
Map

Map(bottom=35465.727294921875, center=[-13.056615445741686, -70.06274475586312], controls=(WidgetControl(optio…

## Conversion de RAW a reflectancia TOA

In [39]:
img_toa = ee.Algorithms.Landsat.TOA(image)

In [40]:
# Simbologia Reflectancia TOA 0 - 1
viz_toa  = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 0.1,
    'max': 0.4
}

In [41]:
type(viz_toa)

dict

In [42]:
# Visualizacion de imagen
Map.addLayer(img_toa, viz_toa, 'toa reflectance')
Map

Map(bottom=35466.0, center=[-13.056615445741686, -70.06274475586312], controls=(WidgetControl(options=['positi…